In [372]:

from gspread_dataframe import get_as_dataframe, set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials
import gspread
import pandas as pd
import numpy as np
import sqlite3
import datetime
from dateutil.relativedelta import relativedelta


DB_PATH = r"C:\python\dataOnline\anaslo_02\db\anaslo_02.db"

spreadSheet_ids = {
    "EXA FIRST": "10-B_vV1pvUzXmvGAiHhODGJgCloOsAmqSO9HvXpk_T8",
    "パラッツォ川越店": "179nJF0NvLng7xPKsd_NX2pJBXsDNsO8SJhOvUAvFk2I",
    }

# 検索キーワードよりホール名取得
SEARCH_WORD = "EXA FIRST"
SPREADSHEET_ID = spreadSheet_ids[SEARCH_WORD]

# スプレッドシート認証設定
scope = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/drive",
]
jsonf = r"C:\python\dataOnline\anaslo_02\json\spreeadsheet-347321-ff675ab5ccbd.json"
creds = ServiceAccountCredentials.from_json_keyfile_name(jsonf, scope)
client = gspread.authorize(creds)
spreadsheet = client.open_by_key(SPREADSHEET_ID)

# # Table name 取得
conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;")
tables = cursor.fetchall()
# print(tables)

cursor.execute(
    "SELECT hall_id, name FROM halls WHERE name LIKE ?", ("%" + SEARCH_WORD + "%",)
)
results = cursor.fetchall()

# 結果表示
if results:
    print(f"🔍 '{SEARCH_WORD}' を含むホール名の検索結果:")
    for hall_id, hall_name in results:
        print(f" - hall_name: {hall_name}, hall_id: {hall_id}")
else:
    print(f"❌ '{SEARCH_WORD}' を含むホール名は見つかりませんでした。")

query = """
-- 出玉データにホール名と機種名を結合して取得
SELECT
    r.*, 
    h.name AS hall_name,     -- ホール名を追加
    m.name AS model_name     -- 機種名を追加
FROM results r
JOIN halls h ON r.hall_id = h.hall_id  -- ホールと結合
JOIN models m ON r.model_id = m.model_id  -- 機種と結合
WHERE h.name = ?  -- 指定ホールのみ
AND m.name LIKE '%ジャグラー%'  -- ジャグラー系機種に限定
ORDER BY r.date DESC, r.unit_no ASC;
"""

# ブドウシミュレーター
def grape_calculator_myfive(game, bb, rb, medals):
    bb_medals = 239.25
    rb_medals = 95.25
    cherry_rate_high = 0.04228
    cherry_rate_low = 0.05847
    replay_rate = 0.411
    denominator_inner = (-medals - (game*3 - (bb*bb_medals + rb*rb_medals + game*replay_rate + game*cherry_rate_high))) / 8
    grape_rate = (game / denominator_inner) - ((game / denominator_inner) * 2)
    
    return grape_rate

df = pd.read_sql_query(query, conn, params=(hall_name,))
conn.close()

df["date"] = pd.to_datetime(df["date"])
df.drop(columns=["result_id", "hall_id", "model_id"], inplace=True)
df = df[["hall_name", "date", "model_name", "unit_no", "game", "BB", "RB", "medals"]]
df["BB_rate"] = (df["game"] / df["BB"]).round(1)
df["RB_rate"] = (df["game"] / df["RB"]).round(1)
df["grape_rate"] = grape_calculator_myfive(df["game"], df["BB"], df["RB"], df["medals"]).round(2)
df["Total_rate"] = (df["game"] / (df["BB"] + df["RB"])).round(1)
df["month"] = df["date"].dt.strftime("%Y-%m")
df["day"] = df["date"].dt.day
df["weekday"] = df["date"].dt.weekday
df["unit_last"] = df["unit_no"].astype(str).str[-1]

df = df.replace([np.inf, -np.inf], np.nan)
df = df.fillna(0)

print(df.shape)
df.head()

🔍 'EXA FIRST' を含むホール名の検索結果:
 - hall_name: EXA FIRST, hall_id: 108
(57456, 16)


,hall_name,date,model_name,unit_no,game,BB,RB,medals,BB_rate,RB_rate,grape_rate,Total_rate,month,day,weekday,unit_last
0,EXA FIRST,2025-04-16,ゴーゴージャグラー3,1001,4838,18,18,-206,268.8,268.8,6.35,134.4,2025-04,16,2,1
1,EXA FIRST,2025-04-16,ゴーゴージャグラー3,1002,2406,8,6,-326,300.8,401.0,5.80,171.9,2025-04,16,2,2
2,EXA FIRST,2025-04-16,ゴーゴージャグラー3,1003,4506,19,13,121,237.2,346.6,6.20,140.8,2025-04,16,2,3
3,EXA FIRST,2025-04-16,ゴーゴージャグラー3,1004,686,1,1,-491,686.0,686.0,5.96,343.0,2025-04,16,2,4
4,EXA FIRST,2025-04-16,ゴーゴージャグラー3,1005,5430,17,19,-762,319.4,285.8,6.04,150.8,2025-04,16,2,5


## RB_RATE

In [365]:
df_tmp = df[(df["model_name"] == "マイジャグラーV")].copy()
df_tmp = df_tmp[(df_tmp["date"].dt.date <= datetime.date(2025, 4, 17)) & (df_tmp["date"].dt.date >= datetime.date(2025, 1, 1))]

game = df_tmp.pivot_table(index=["model_name", "unit_no"],columns="date",values="game",aggfunc="sum",margins=True,margins_name="TOTAL")
bb = df_tmp.pivot_table(index=["model_name", "unit_no"],columns="date",values="BB",aggfunc="sum",margins=True,margins_name="TOTAL",)
rb = df_tmp.pivot_table(index=["model_name", "unit_no"],columns="date",values="RB",aggfunc="sum",margins=True,margins_name="TOTAL",)
medals = df_tmp.pivot_table(index=["model_name", "unit_no"], columns="date", values="medals", aggfunc="sum")


In [366]:
rb_rate = (game / rb).round(1)
rb_rate.to_csv("rb_rate.csv", encoding="utf_8_sig")
rb_rate.sort_values(by="TOTAL", ascending=True, inplace=True)
rb_rate.head()

date                2025-01-01 00:00:00  2025-01-02 00:00:00  \
model_name unit_no                                             
マイジャグラーV   1017                   322.9                596.4   
           1047                   309.0                424.9   
           1039                   205.5                787.0   
           1078                   288.7                224.9   
           1034                   217.5                544.2   

date                2025-01-03 00:00:00  2025-01-04 00:00:00  \
model_name unit_no                                             
マイジャグラーV   1017                   622.0                340.6   
           1047                   502.4                623.8   
           1039                   704.0                319.7   
           1078                   320.2                589.2   
           1034                   488.4                243.3   

date                2025-01-05 00:00:00  2025-01-06 00:00:00  \
model_name unit_no                                             
マイジャグラーV   1017                   298.1                301.7   
           1047                   213.1                  inf   
           1039                   449.6                282.8   
           1078                   277.8                420.5   
           1034                   257.5                318.0   

date                2025-01-07 00:00:00  2025-01-08 00:00:00  \
model_name unit_no                                             
マイジャグラーV   1017                   235.4                542.8   
           1047                   255.7                254.3   
           1039                   707.2                347.0   
           1078                   400.1                293.5   
           1034                   289.5                334.9   

date                2025-01-09 00:00:00  2025-01-10 00:00:00  \
model_name unit_no                                             
マイジャグラーV   1017                   360.5                636.2   
           1047                   278.4                300.6   
           1039                   482.2                280.8   
           1078                  1078.0                  inf   
           1034                   523.2                257.3   

date                2025-01-11 00:00:00  2025-01-12 00:00:00  \
model_name unit_no                                             
マイジャグラーV   1017                   502.6                214.0   
           1047                   344.5                391.9   
           1039                   322.7                326.3   
           1078                   260.6                237.1   
           1034                   657.5                835.5   

date                2025-01-13 00:00:00  2025-01-14 00:00:00  \
model_name unit_no                                             
マイジャグラーV   1017                   417.3                352.8   
           1047                   457.0                402.5   
           1039                   557.5                311.1   
           1078                   408.2                360.7   
           1034                   330.4                472.0   

date                2025-01-15 00:00:00  2025-01-16 00:00:00  \
model_name unit_no                                             
マイジャグラーV   1017                   335.8                363.0   
           1047                   296.9                310.4   
           1039                   797.0                  inf   
           1078                   398.0                294.6   
           1034                   373.9                292.4   

date                2025-01-17 00:00:00  2025-01-18 00:00:00  \
model_name unit_no                                             
マイジャグラーV   1017                   302.5                389.7   
           1047                   176.8                326.1   
           1039                  1335.0                487.8   
           1078                   295.4                329.2   
           1034         

In [367]:
total_rate = (game / (bb+rb)).round(1)
total_rate.to_csv("total_rate.csv", encoding="utf_8_sig")
total_rate.head()

date                2025-01-01 00:00:00  2025-01-02 00:00:00  \
model_name unit_no                                             
マイジャグラーV   1001                   113.8                150.7   
           1002                   129.0                160.8   
           1003                   151.6                150.0   
           1004                   163.5                150.7   
           1005                   147.1                175.8   

date                2025-01-03 00:00:00  2025-01-04 00:00:00  \
model_name unit_no                                             
マイジャグラーV   1001                   189.8                133.0   
           1002                   148.4                232.8   
           1003                   146.5                127.6   
           1004                   177.6                169.7   
           1005                   290.4                137.5   

date                2025-01-05 00:00:00  2025-01-06 00:00:00  \
model_name unit_no                                             
マイジャグラーV   1001                   168.0                171.9   
           1002                   194.6                243.0   
           1003                   162.8                295.6   
           1004                   174.0                148.5   
           1005                   240.9                136.3   

date                2025-01-07 00:00:00  2025-01-08 00:00:00  \
model_name unit_no                                             
マイジャグラーV   1001                   214.7                158.2   
           1002                   148.3                220.2   
           1003                   166.4                143.8   
           1004                   148.8                431.0   
           1005                   144.1                155.3   

date                2025-01-09 00:00:00  2025-01-10 00:00:00  \
model_name unit_no                                             
マイジャグラーV   1001                   139.5                179.0   
           1002                   132.4                149.4   
           1003                   193.9                219.6   
           1004                   415.5                158.3   
           1005                   206.0                316.5   

date                2025-01-11 00:00:00  2025-01-12 00:00:00  \
model_name unit_no                                             
マイジャグラーV   1001                   197.0                116.5   
           1002                   142.2                184.9   
           1003                   299.7                174.3   
           1004                   108.7                173.7   
           1005                   126.1                198.7   

date                2025-01-13 00:00:00  2025-01-14 00:00:00  \
model_name unit_no                                             
マイジャグラーV   1001                   125.2                189.2   
           1002                   199.2                110.0   
           1003                   118.7                170.3   
           1004                   132.5                183.5   
           1005                   163.7                152.1   

date                2025-01-15 00:00:00  2025-01-16 00:00:00  \
model_name unit_no                                             
マイジャグラーV   1001                   174.6                156.9   
           1002                   130.1                250.2   
           1003                   264.3                232.8   
           1004                   121.3                668.0   
           1005                   161.5                217.1   

date                2025-01-17 00:00:00  2025-01-18 00:00:00  \
model_name unit_no                                             
マイジャグラーV   1001                   198.0                137.5   
           1002                   180.7                161.8   
           1003                   142.5                133.1   
           1004                   149.2                200.8   
           1005         

In [368]:
game.to_csv("game.csv", encoding="utf_8_sig")
game.head()

date                2025-01-01 00:00:00  2025-01-02 00:00:00  \
model_name unit_no                                             
マイジャグラーV   1001                  6939.0               4371.0   
           1002                  6193.0               1608.0   
           1003                  3336.0               5249.0   
           1004                  4414.0               5274.0   
           1005                  4412.0               4395.0   

date                2025-01-03 00:00:00  2025-01-04 00:00:00  \
model_name unit_no                                             
マイジャグラーV   1001                  1898.0               6384.0   
           1002                  7718.0                931.0   
           1003                  4687.0               8421.0   
           1004                  3019.0               2545.0   
           1005                  2033.0               5361.0   

date                2025-01-05 00:00:00  2025-01-06 00:00:00  \
model_name unit_no                                             
マイジャグラーV   1001                  7561.0               2063.0   
           1002                  5060.0               1701.0   
           1003                  4559.0               1478.0   
           1004                  5743.0               4159.0   
           1005                  4095.0               7770.0   

date                2025-01-07 00:00:00  2025-01-08 00:00:00  \
model_name unit_no                                             
マイジャグラーV   1001                  1503.0               3956.0   
           1002                  1631.0               1101.0   
           1003                  2663.0               6184.0   
           1004                  6844.0                431.0   
           1005                  6483.0               2950.0   

date                2025-01-09 00:00:00  2025-01-10 00:00:00  \
model_name unit_no                                             
マイジャグラーV   1001                  3209.0               1432.0   
           1002                  5694.0               5677.0   
           1003                  1745.0               1098.0   
           1004                   831.0               3799.0   
           1005                   412.0                633.0   

date                2025-01-11 00:00:00  2025-01-12 00:00:00  \
model_name unit_no                                             
マイジャグラーV   1001                  7290.0               6873.0   
           1002                  7392.0               2588.0   
           1003                  5095.0               6099.0   
           1004                  7721.0               3995.0   
           1005                  6555.0               2384.0   

date                2025-01-13 00:00:00  2025-01-14 00:00:00  \
model_name unit_no                                             
マイジャグラーV   1001                  7639.0               1514.0   
           1002                   996.0               7152.0   
           1003                  7360.0               1533.0   
           1004                  7157.0                367.0   
           1005                  2292.0               3194.0   

date                2025-01-15 00:00:00  2025-01-16 00:00:00  \
model_name unit_no                                             
マイジャグラーV   1001                  5062.0               4863.0   
           1002                  7415.0               1251.0   
           1003                  1850.0               1397.0   
           1004                  6794.0                668.0   
           1005                  4684.0               1954.0   

date                2025-01-17 00:00:00  2025-01-18 00:00:00  \
model_name unit_no                                             
マイジャグラーV   1001                  4950.0               5773.0   
           1002                  1265.0               4530.0   
           1003                  7269.0               4259.0   
           1004                  5520.0               1606.0   
           1005         

In [369]:
medals.to_csv("medals.csv", encoding="utf_8_sig")
medals.tail()

date                2025-01-01  2025-01-02  2025-01-03  2025-01-04  \
model_name unit_no                                                   
マイジャグラーV   1095           44.0       771.0      -571.0       -32.0   
           1096          -62.0      2974.0     -1453.0       465.0   
           1097        -1712.0      -421.0      1615.0      -109.0   
           1098         2129.0      -744.0      -241.0     -1029.0   
           1099         2444.0       265.0       -88.0      -503.0   

date                2025-01-05  2025-01-06  2025-01-07  2025-01-08  \
model_name unit_no                                                   
マイジャグラーV   1095          947.0      -291.0      1756.0      -426.0   
           1096         -653.0       288.0       394.0      -468.0   
           1097         -429.0      -544.0       453.0       232.0   
           1098         -726.0      1006.0       156.0      -415.0   
           1099         -844.0      -759.0       335.0      -244.0   

date                2025-01-09  2025-01-10  2025-01-11  2025-01-12  \
model_name unit_no                                                   
マイジャグラーV   1095         2506.0      -191.0      1606.0      -409.0   
           1096         -309.0      1088.0      1826.0        76.0   
           1097         -762.0      -679.0      1544.0       194.0   
           1098         -321.0       253.0     -1985.0     -1076.0   
           1099         -797.0      1088.0     -1265.0     -1288.0   

date                2025-01-13  2025-01-14  2025-01-15  2025-01-16  \
model_name unit_no                                                   
マイジャグラーV   1095         -582.0       438.0      -803.0      -215.0   
           1096         -368.0      -115.0       362.0      -371.0   
           1097          679.0      -191.0      -782.0      -285.0   
           1098         -700.0      1353.0     -1100.0      -232.0   
           1099          638.0      -256.0     -1724.0      -185.0   

date                2025-01-17  2025-01-18  2025-01-19  2025-01-20  \
model_name unit_no                                                   
マイジャグラーV   1095         -332.0      -979.0      1579.0      -668.0   
           1096         -944.0      -712.0      -959.0      -638.0   
           1097          279.0      -368.0      -626.0      -385.0   
           1098         1076.0      -568.0     -1935.0      -279.0   
           1099         -656.0      -732.0     -1006.0       747.0   

date                2025-01-21  2025-01-22  2025-01-23  2025-01-24  \
model_name unit_no                                                   
マイジャグラーV   1095         -103.0       771.0       853.0      -750.0   
           1096        -1029.0       244.0      -597.0       700.0   
           1097         2647.0      -115.0        15.0     -1241.0   
           1098         -921.0      -194.0       109.0      -315.0   
           1099         -609.0       744.0       821.0      -109.0   

date                2025-01-25  2025-01-26  2025-01-27  2025-01-28  \
model_name unit_no                                                   
マイジャグラーV   1095         1041.0     -1841.0     -1406.0      1688.0   
           1096         2126.0     -1194.0      -262.0      -368.0   
           1097         3071.0      1932.0         3.0      -285.0   
           1098         3282.0     -1700.0       221.0      -606.0   
           1099         2868.0     -2744.0     -1618.0      -115.0   

date                2025-01-29  2025-01-30  2025-01-31  2025-02-01  \
model_name unit_no                                                   
マイジャグラーV   1095         -309.0       688.0      1474.0        76.0   
           1096          759.0       441.0     -1265.0      -794.0   
           1097         -785.0       312.0      2585.0      -338.0   
           1098         -268.0      -850.0      3956.0     -1441.0   
           1099         1606.0      -132.0      -838.0      -644.0   

date                2025-02-02  2025-02-03  2025-02-04  2025-02-05  \
mo

In [371]:
cumulative_df = medals.iloc[:, ::-1].cumsum(axis=1).iloc[:, ::-1]
cumulative_df = medals.cumsum(axis=1)
cumulative_df.to_csv("cumulative_df.csv", encoding="utf_8_sig")
cumulative_df.tail()

date                2025-01-01  2025-01-02  2025-01-03  2025-01-04  \
model_name unit_no                                                   
マイジャグラーV   1095           44.0       815.0       244.0       212.0   
           1096          -62.0      2912.0      1459.0      1924.0   
           1097        -1712.0     -2133.0      -518.0      -627.0   
           1098         2129.0      1385.0      1144.0       115.0   
           1099         2444.0      2709.0      2621.0      2118.0   

date                2025-01-05  2025-01-06  2025-01-07  2025-01-08  \
model_name unit_no                                                   
マイジャグラーV   1095         1159.0       868.0      2624.0      2198.0   
           1096         1271.0      1559.0      1953.0      1485.0   
           1097        -1056.0     -1600.0     -1147.0      -915.0   
           1098         -611.0       395.0       551.0       136.0   
           1099         1274.0       515.0       850.0       606.0   

date                2025-01-09  2025-01-10  2025-01-11  2025-01-12  \
model_name unit_no                                                   
マイジャグラーV   1095         4704.0      4513.0      6119.0      5710.0   
           1096         1176.0      2264.0      4090.0      4166.0   
           1097        -1677.0     -2356.0      -812.0      -618.0   
           1098         -185.0        68.0     -1917.0     -2993.0   
           1099         -191.0       897.0      -368.0     -1656.0   

date                2025-01-13  2025-01-14  2025-01-15  2025-01-16  \
model_name unit_no                                                   
マイジャグラーV   1095         5128.0      5566.0      4763.0      4548.0   
           1096         3798.0      3683.0      4045.0      3674.0   
           1097           61.0      -130.0      -912.0     -1197.0   
           1098        -3693.0     -2340.0     -3440.0     -3672.0   
           1099        -1018.0     -1274.0     -2998.0     -3183.0   

date                2025-01-17  2025-01-18  2025-01-19  2025-01-20  \
model_name unit_no                                                   
マイジャグラーV   1095         4216.0      3237.0      4816.0      4148.0   
           1096         2730.0      2018.0      1059.0       421.0   
           1097         -918.0     -1286.0     -1912.0     -2297.0   
           1098        -2596.0     -3164.0     -5099.0     -5378.0   
           1099        -3839.0     -4571.0     -5577.0     -4830.0   

date                2025-01-21  2025-01-22  2025-01-23  2025-01-24  \
model_name unit_no                                                   
マイジャグラーV   1095         4045.0      4816.0      5669.0      4919.0   
           1096         -608.0      -364.0      -961.0      -261.0   
           1097          350.0       235.0       250.0      -991.0   
           1098        -6299.0     -6493.0     -6384.0     -6699.0   
           1099        -5439.0     -4695.0     -3874.0     -3983.0   

date                2025-01-25  2025-01-26  2025-01-27  2025-01-28  \
model_name unit_no                                                   
マイジャグラーV   1095         5960.0      4119.0      2713.0      4401.0   
           1096         1865.0       671.0       409.0        41.0   
           1097         2080.0      4012.0      4015.0      3730.0   
           1098        -3417.0     -5117.0     -4896.0     -5502.0   
           1099        -1115.0     -3859.0     -5477.0     -5592.0   

date                2025-01-29  2025-01-30  2025-01-31  2025-02-01  \
model_name unit_no                                                   
マイジャグラーV   1095         4092.0      4780.0      6254.0      6330.0   
           1096          800.0      1241.0       -24.0      -818.0   
           1097         2945.0      3257.0      5842.0      5504.0   
           1098        -5770.0     -6620.0     -2664.0     -4105.0   
           1099        -3986.0     -4118.0     -4956.0     -5600.0   

date                2025-02-02  2025-02-03  2025-02-04  2025-02-05  \
mo

In [ ]:
condition = df["date"].dt.date >= datetime.date.today() - relativedelta(days=1)
df["date"][condition].unique()
# print(df["model_name"].unique())
# print(df["model_name"][condition].unique())
# print(df["unit_no"].nunique())
# print(df["unit_no"][condition].nunique())


['マイジャグラーV' 'ファンキージャグラー2' 'ミスタージャグラー' 'ゴーゴージャグラー3' 'ウルトラミラクルジャグラー'
 'アイムジャグラーEX-TP' 'ジャグラーガールズ' 'ハッピージャグラーVIII' 'SミスタージャグラーKK']
['マイジャグラーV' 'ファンキージャグラー2' 'ミスタージャグラー' 'ゴーゴージャグラー3' 'ウルトラミラクルジャグラー'
 'アイムジャグラーEX-TP' 'ジャグラーガールズ']
152
152


## MONTH_ANALYSIS

In [10]:
# 日ごとにモデル別で表示
SHEET_NAME = "HOLE_ANALYSIS"
MODEL_NAME = "マイジャグラーV"
df_tmp = df[(df["model_name"] == MODEL_NAME)].copy()
# メダル
summary = df_tmp.pivot_table(
    index=["model_name", "unit_no"],
    columns="month",
    values="medals",
    aggfunc="sum",
    margins=True,
    margins_name="Total",
)
summary.head()

month               2024-04  2024-05  2024-06  2024-07  2024-08  2024-09  \
model_name unit_no                                                         
マイジャグラーV   1001      2895.0   4620.0   1012.0  -7425.0  -3903.0   3233.0   
           1002     13950.0   5351.0   8095.0   7651.0   2323.0   8350.0   
           1003     14057.0  12475.0   9104.0   4498.0   1771.0   6580.0   
           1004      4949.0   8723.0   3594.0   6261.0   4198.0   -907.0   
           1005     11637.0   4167.0  11100.0   5600.0    820.0     51.0   

month               2024-10  2024-11  2024-12  2025-01  2025-02  2025-03  \
model_name unit_no                                                         
マイジャグラーV   1001      2371.0  -7162.0  -9032.0   1606.0 -10442.0  -7233.0   
           1002      5227.0  -7894.0    591.0  13541.0  -1870.0   5959.0   
           1003      -877.0   8672.0   6747.0  -4302.0   4553.0   5974.0   
           1004      7221.0   1500.0  -2992.0   8438.0   4868.0   5599.0   
           1005     17447.0  -2571.0  -3268.0   5252.0    362.0   1950.0   

month               2025-04  Total  
model_name unit_no                  
マイジャグラーV   1001     -1672.0 -31132  
           1002      3149.0  64423  
           1003      4924.0  74176  
           1004     -3734.0  47718  
           1005      6048.0  58595

## HOLE_ANALYSIS

In [7]:
# 日ごとにモデル別で表示
SHEET_NAME = "HOLE_ANALYSIS"
df_tmp = df.copy()
# メダル
medals = df_tmp.pivot_table(
    index=["model_name"],
    columns="day",
    values="medals",
    aggfunc="sum",
    margins=True,
    margins_name="Total",
)
medals.columns = pd.MultiIndex.from_product([["MEDALS"], medals.columns])
# RB＿RATE
game = df_tmp.pivot_table(
    index=["model_name"],
    columns="day",
    values="game",
    aggfunc="sum",
    margins=True,
    margins_name="Total",
)
rb = df_tmp.pivot_table(
    index=["model_name"],
    columns="day",
    values="RB",
    aggfunc="sum",
    margins=True,
    margins_name="Total",
)
rb_rate = (game / rb).round(1)
rb_rate.columns = pd.MultiIndex.from_product([["RB_RATE"], rb_rate.columns])
rb_rate = rb_rate.replace([np.inf, -np.inf], np.nan)
rb_rate = rb_rate.fillna("")
game.columns = pd.MultiIndex.from_product([["GAME"], game.columns])
# WRITE
sheet = spreadsheet.worksheet(SHEET_NAME)
sheet.clear()
set_with_dataframe(sheet, medals, include_index=True)
existing = get_as_dataframe(sheet, evaluate_formulas=True)
next_row = len(existing) + 4  # +2: ヘッダー + 次の空行
set_with_dataframe(sheet, rb_rate, row=next_row, include_index=True)
existing = get_as_dataframe(sheet, evaluate_formulas=True)
next_row = len(existing) + 6  # +2: ヘッダー + 次の空行
set_with_dataframe(sheet, game, row=next_row, include_index=True)

## UNIT_ANALYSIS

In [ ]:
# マシン別メダル
SHEET_NAME = "UNIT_ANALYSIS"
MACHINE_NAME = "マイジャグラーV"
# df_tmp = df[df["model_name"] == MACHINE_NAME].copy()
df_tmp = df.copy()
# メダル数
medals = df_tmp.pivot_table(
    index=["model_name", "unit_no"],
    columns="day",
    values="medals",
    aggfunc="sum",
    margins=True,
    margins_name="Total",
)
# medals["rank"] = medals["Total"].rank(method="min", ascending=False).astype(int)
medals.columns = pd.MultiIndex.from_product([["MEDALS"], medals.columns])
medals.shape
# RB_rate 作成
game = df_tmp.pivot_table(
    index=["model_name", "unit_no"],
    columns="day",
    values="game",
    aggfunc="sum",
    margins=True,
    margins_name="Total",
)
rb = df_tmp.pivot_table(
    index=["model_name", "unit_no"],
    columns="day",
    values="RB",
    aggfunc="sum",
    margins=True,
    margins_name="Total",
)
rb_rate = (game / rb).round(1)
# rb_rate["rank"] = rb_rate["Total"].rank(method="min", ascending=True).astype(int)
rb_rate.columns = pd.MultiIndex.from_product([["RB_RATE"], rb_rate.columns])
rb_rate = rb_rate.replace([np.inf, -np.inf], np.nan)
rb_rate = rb_rate.fillna(0)
rb_rate.shape

# 回転数
games = df_tmp.pivot_table(
    index=["model_name", "unit_no"],
    columns="day",
    values="game",
    aggfunc="sum",
    margins=True,
    margins_name="Total",
)
# games["rank"] = games["Total"].rank(method="min", ascending=False).astype(int)
games.columns = pd.MultiIndex.from_product([["GAMES"], games.columns])
games.head()

next_row = 1
spreadsheet = client.open_by_key(SPREADSHEET_ID)
sheet = spreadsheet.worksheet(SHEET_NAME)
sheet.clear()
set_with_dataframe(sheet, medals, include_index=True)
existing = get_as_dataframe(sheet, evaluate_formulas=True)
next_row = len(existing) + 4  # +2: ヘッダー + 次の空行
set_with_dataframe(sheet, rb_rate, row=next_row, include_index=True)
existing = get_as_dataframe(sheet, evaluate_formulas=True)
next_row = len(existing) + 6  # +2: ヘッダー + 次の空行
set_with_dataframe(sheet, games, row=next_row, include_index=True)
existing = get_as_dataframe(sheet, evaluate_formulas=True)

## DAY_DETAIL_N

In [ ]:
# 日付指定
DAY = 15
SHEET_NAME = f"DAY_DETAIL_{DAY}"

next_row = 1
spreadsheet = client.open_by_key(SPREADSHEET_ID)
sheet = spreadsheet.worksheet(SHEET_NAME)

models = [
    "マイジャグラーV",
    # "ゴーゴージャグラー3",
    # "アイムジャグラーEX-TP",
    # "ファンキージャグラー2",
]

for model in models:

    df_tmp = df[(df["day"] == DAY) & (df["model_name"] == model)].copy()
    df_tmp["date"] = df_tmp["date"].dt.strftime("%Y-%m-%d")

    # メダル数
    desired_order = ["medals", "game", "Total_rate", "RB_rate", "grape_rate"]
    medals = df_tmp.pivot_table(
        index=["model_name", "unit_no"],
        columns="date",
        values=desired_order,
        aggfunc="sum",
        # margins=True,
        # margins_name="Total",
    )
    # カラム順序を入れ替える（swaplevel）＆ ソート
    medals = medals.replace([np.inf, -np.inf], np.nan)
    medals = medals.fillna(0)
    medals.columns = medals.columns.swaplevel(0, 1)
    medals = medals.sort_index(axis=1, level=0, ascending=False)  # 日付ごとに並び替え

    # medals.drop(labels="Total", level=0, inplace=True)
    # medals.columns = pd.MultiIndex.from_product([["MEDALS"], medals.columns])
    display(medals.head())

    # RB_RATE
    # game = df_tmp.pivot_table(
    #     index=["model_name", "unit_no"],
    #     columns="date",
    #     values="game",
    #     aggfunc="sum",
    #     margins=True,
    #     margins_name="Total",
    # )
    # rb = df_tmp.pivot_table(
    #     index=["model_name", "unit_no"],
    #     columns="date",
    #     values="RB",
    #     aggfunc="sum",
    #     margins=True,
    #     margins_name="Total",
    # )
    
    # rb_rate = (game / rb).round(1)
    # medals.drop(labels="Total", level=0, inplace=True)
    # rb_rate["rank"] = rb_rate["Total"].rank(method="min", ascending=True).astype(int)
    # rb_rate.columns = pd.MultiIndex.from_product([["RB_RATE"], rb_rate.columns])
    # rb_rate = rb_rate.replace([np.inf, -np.inf], np.nan)
    # rb_rate = rb_rate.fillna(0)
    # print(f"{model}_{rb_rate.shape}")

    # スプレッドシート追記
    sheet.clear()
    set_with_dataframe(sheet, medals, row=next_row, include_index=True)
    existing = get_as_dataframe(sheet, evaluate_formulas=True)
    next_row += medals.shape[0] + 5  # 追記行
    # # set_with_dataframe(sheet, rb_rate, row=next_row, include_index=True)
    # existing = get_as_dataframe(sheet, evaluate_formulas=True)
    # next_row += medals.shape[0] + 5  # 追記行

In [ ]:
medals.columns

## DAY n

In [373]:
def medals_and_rb_rate(machine_name, year, month, day, period=3):
    base_date = datetime.date(year, month, day)
    start_date = base_date + datetime.timedelta(days=-1)
    end_date = base_date + datetime.timedelta(days=-period)
    df_tmp = df[
        (df["model_name"] == machine_name)
        & (df["date"].dt.date <= start_date)
        & (df["date"].dt.date >= end_date)
    ]

    # メダル
    medals = df_tmp.pivot_table(
        index=["model_name", "unit_no"],
        columns="date",
        values="medals",
        aggfunc="sum",
        margins=True,
        margins_name="Total",
    )
    medals.drop(labels="Total", level=0, inplace=True)
    medals["rank"] = medals["Total"].rank(method="min", ascending=True).astype(int)
    medals.columns = pd.MultiIndex.from_product([["MEDALS"], medals.columns])
    medals.tail()

    # RB_RATE
    df_tmp = df[(df["model_name"] == machine_name) & (df["date"].dt.date == base_date)]
    game = df_tmp.pivot_table(
        index=["model_name", "unit_no"],
        columns="date",
        values="game",
        aggfunc="sum",
        # margins=True,
        # margins_name="Total",
    )
    bb = df_tmp.pivot_table(
        index=["model_name", "unit_no"],
        columns="date",
        values="BB",
        aggfunc="sum",
        # margins=True,
        # margins_name="Total",
    )
    rb = df_tmp.pivot_table(
        index=["model_name", "unit_no"],
        columns="date",
        values="RB",
        aggfunc="sum",
        # margins=True,
        # margins_name="Total",
    )
    rb_rate = (game / rb).round(1)
    rb_rate.columns = pd.MultiIndex.from_product([["RB_RATE"], rb_rate.columns])
    rb_rate.head()
    
    total_rate = (game / (bb+rb)).round(1)
    total_rate.columns = pd.MultiIndex.from_product([["TOTAL_RATE"], total_rate.columns])

    game.columns = pd.MultiIndex.from_product([["GAME"], game.columns])
    bb.columns = pd.MultiIndex.from_product([["BB"], bb.columns])
    rb.columns = pd.MultiIndex.from_product([["RB"], rb.columns])

    # データを表示（長い場合は全表示）
    pd.set_option("display.max_rows", None)
    pd.set_option("display.max_columns", None)
    medal_rb_rate = pd.concat([medals, game, total_rate, rb_rate, bb, rb], axis=1)
    medal_rb_rate = medal_rb_rate.replace([np.inf, -np.inf], np.nan)
    medal_rb_rate = medal_rb_rate.fillna(0)

    medal_rb_rate = medal_rb_rate[medal_rb_rate["MEDALS"]["rank"] <= 10]

    return medal_rb_rate


# N_DAYS_AGO
MACHINE_NAME = "マイジャグラーV"

medal_rb_rate = medals_and_rb_rate(MACHINE_NAME, 2025, 4, 14)
medal_rb_rate.head(10)

MEDALS                      \
date               2025-04-11 00:00:00 2025-04-12 00:00:00   
model_name unit_no                                           
マイジャグラーV   1001                   -453               -1112   
           1029                  -1371                -924   
           1042                  -1844               -1579   
           1049                   -782                -782   
           1050                  -1006                -935   
           1069                  -2515                -356   
           1082                  -1512                -276   
           1083                  -2568                1088   
           1089                  -1135               -1582   
           1091                  -1135                -915   

                                                                 GAME  \
date               2025-04-13 00:00:00 Total rank 2025-04-14 00:00:00   
model_name unit_no                                                      
マイジャグラーV   1001                   -562 -2127   10                 0.0   
           1029                  -1547 -3842    1              1781.0   
           1042                   -291 -3714    2              7089.0   
           1049                   -571 -2135    9              2363.0   
           1050                   -974 -2915    5              8567.0   
           1069                   -315 -3186    4              6827.0   
           1082                   -488 -2276    8              8576.0   
           1083                   -865 -2345    7              8189.0   
           1089                   -726 -3443    3              6668.0   
           1091                   -562 -2612    6              4896.0   

                            TOTAL_RATE             RB_RATE  \
date               2025-04-14 00:00:00 2025-04-14 00:00:00   
model_name unit_no                                           
マイジャグラーV   1001                    0.0                 0.0   
           1029                  445.2                 0.0   
           1042                  133.8               244.4   
           1049                  181.8               787.7   
           1050                  153.0               342.7   
           1069                  170.7               525.2   
           1082                  138.3               295.7   
           1083                  117.0               264.2   
           1089                  155.1               350.9   
           1091                  148.4               349.7   

                                    BB                  RB  
date               2025-04-14 00:00:00 2025-04-14 00:00:00  
model_name unit_no                                          
マイジャグラーV   1001                    0.0                 0.0  
           1029                    4.0                 0.0  
           1042                   24.0                29.0  
           1049                   10.0                 3.0  
           1050                   31.0                25.0  
           1069                   27.0                13.0  
           1082                   33.0                29.0  
           1083                   39.0                31.0  
           1089                   24.0                19.0  
           1091                   19.0                14.0

In [374]:
models = ["マイジャグラーV", "ゴーゴージャグラー3", "アイムジャグラーEX-TP", "ファンキージャグラー2"]
today = datetime.date.today()
spreadsheet = client.open_by_key(SPREADSHEET_ID)
# スプレッドシート追記
target_day = 1
for target_day in range(today.day-1, today.day+1):
    sheet_name = f"DAY{target_day}"
    sheet = spreadsheet.worksheet(sheet_name)
    sheet.clear()
    base = datetime.date(today.year, today.month, target_day)
    dates = [base - relativedelta(months=i) for i in range(7)]
    
    next_row = 1
    for model in models:
        for d in dates:
            medal_rb_rate = medals_and_rb_rate(model, d.year, d.month, d.day)
            set_with_dataframe(sheet, medal_rb_rate, row=next_row, include_index=True)
            existing = get_as_dataframe(sheet, evaluate_formulas=True)
            next_row += medal_rb_rate.shape[0] + 5

    print(f"DAY{target_day} 書き込み完了しました")

DAY16 書き込み完了しました


APIError: APIError: [429]: Quota exceeded for quota metric 'Write requests' and limit 'Write requests per minute per user' of service 'sheets.googleapis.com' for consumer 'project_number:660712279213'.

## MEDALS_nDAYS_AGO

In [391]:
def get_medals_summary(start_date, end_date, model_name):
    # RB_RATE
    df_tmp = df[(df["model_name"] == model_name) & (df["date"].dt.date <= start_date) & (df["date"].dt.date >= end_date)].copy()

    medals = df_tmp.pivot_table(index=["model_name", "unit_no"], columns="date", values="medals", aggfunc="sum")
    # medals = medals.iloc[:, ::-1].cumsum(axis=1).iloc[:, ::-1]
    # medals.drop(labels="Total", level=0, inplace=True)
    # medals["Rank"] = medals["Total"].rank(method="min", ascending=True).astype(int)
    # medals.columns = pd.MultiIndex.from_product([["MEDALS"], medals.columns])
    term = 7
    for i in range(term):
        medals[f"total{term-i}"] = medals.iloc[:, i:term].sum(axis=1)
    for i in range(term):
        medals[f"rank{term-i}"] = medals[f"total{term-i}"].rank(method="min", ascending=True).astype(int)

    return medals


today = datetime.date.today()
start_date = today + datetime.timedelta(days=-4)
end_date = today + datetime.timedelta(days=-10)
ndays_ago = get_medals_summary(start_date, end_date, model)
ndays_ago.columns = pd.MultiIndex.from_product([["ndays_ago"], ndays_ago.columns])
ndays_ago.tail()

ndays_ago                      \
date               2025-04-07 00:00:00 2025-04-08 00:00:00   
model_name unit_no                                           
マイジャグラーV   1095                    912                -485   
           1096                     26                -274   
           1097                  -1076                2109   
           1098                     91                -368   
           1099                   -615                -359   

                                                            \
date               2025-04-09 00:00:00 2025-04-10 00:00:00   
model_name unit_no                                           
マイジャグラーV   1095                    279                  97   
           1096                   -650                -456   
           1097                   -285                 971   
           1098                    -62                  88   
           1099                   1112                 226   

                                                            \
date               2025-04-11 00:00:00 2025-04-12 00:00:00   
model_name unit_no                                           
マイジャグラーV   1095                   -656                -403   
           1096                  -1253                 -74   
           1097                   -644                 268   
           1098                   2797                 185   
           1099                   3165                2765   

                                                                           \
date               2025-04-13 00:00:00 total7 total6 total5 total4 total3   
model_name unit_no                                                          
マイジャグラーV   1095                   -385   -641  -1553  -1068  -1347  -1444   
           1096                   -174  -2855  -2881  -2607  -1957  -1501   
           1097                  -1288     55   1131   -978   -693  -1664   
           1098                   -809   1922   1831   2199   2261   2173   
           1099                   -426   5868   6483   6842   5730   5504   

                                                                            
date               total2 total1 rank7 rank6 rank5 rank4 rank3 rank2 rank1  
model_name unit_no                                                          
マイジャグラーV   1095      -788   -385    38    22    28    25    25    33    43  
           1096      -248   -174    10    10     6    14    23    45    55  
           1097     -1020  -1288    47    59    31    32    18    23     3  
           1098      -624   -809    66    63    67    69    65    38    13  
           1099      2339   -426    80    81    80    80    80    77    39

In [392]:
model_name = "マイジャグラーV"
df_tmp = df[(df["model_name"] == model_name) & (df["date"].dt.date == today+datetime.timedelta(days=-3))]
game = df_tmp.pivot_table(index=["model_name", "unit_no"], columns="date", values="game", aggfunc="sum")
bb = df_tmp.pivot_table(index=["model_name", "unit_no"], columns="date", values="BB", aggfunc="sum")
rb = df_tmp.pivot_table(index=["model_name", "unit_no"], columns="date", values="RB", aggfunc="sum")
rb_rate = (game / rb).round(1)
total_rate = (game / (bb+rb)).round(1)
rb_rate.columns = pd.MultiIndex.from_product([["RB_RATE"], rb_rate.columns])
total_rate.columns = pd.MultiIndex.from_product([["TOTAL_RATE"], total_rate.columns])
game.columns = pd.MultiIndex.from_product([["GAME"], game.columns])
bb.columns = pd.MultiIndex.from_product([["BB"], bb.columns])
rb.columns = pd.MultiIndex.from_product([["RB"], rb.columns])
medal_rb_rate = pd.concat([ndays_ago, game, bb, rb, total_rate, rb_rate], axis=1)
medal_rb_rate.to_csv(f"{model_name}_medal_rb_rate.csv", encoding="utf_8_sig")
medal_rb_rate.tail()

ndays_ago                      \
date               2025-04-07 00:00:00 2025-04-08 00:00:00   
model_name unit_no                                           
マイジャグラーV   1095                    912                -485   
           1096                     26                -274   
           1097                  -1076                2109   
           1098                     91                -368   
           1099                   -615                -359   

                                                            \
date               2025-04-09 00:00:00 2025-04-10 00:00:00   
model_name unit_no                                           
マイジャグラーV   1095                    279                  97   
           1096                   -650                -456   
           1097                   -285                 971   
           1098                    -62                  88   
           1099                   1112                 226   

                                                            \
date               2025-04-11 00:00:00 2025-04-12 00:00:00   
model_name unit_no                                           
マイジャグラーV   1095                   -656                -403   
           1096                  -1253                 -74   
           1097                   -644                 268   
           1098                   2797                 185   
           1099                   3165                2765   

                                                                           \
date               2025-04-13 00:00:00 total7 total6 total5 total4 total3   
model_name unit_no                                                          
マイジャグラーV   1095                   -385   -641  -1553  -1068  -1347  -1444   
           1096                   -174  -2855  -2881  -2607  -1957  -1501   
           1097                  -1288     55   1131   -978   -693  -1664   
           1098                   -809   1922   1831   2199   2261   2173   
           1099                   -426   5868   6483   6842   5730   5504   

                                                                            \
date               total2 total1 rank7 rank6 rank5 rank4 rank3 rank2 rank1   
model_name unit_no                                                           
マイジャグラーV   1095      -788   -385    38    22    28    25    25    33    43   
           1096      -248   -174    10    10     6    14    23    45    55   
           1097     -1020  -1288    47    59    31    32    18    23     3   
           1098      -624   -809    66    63    67    69    65    38    13   
           1099      2339   -426    80    81    80    80    80    77    39   

                                  GAME                  BB  \
date               2025-04-14 00:00:00 2025-04-14 00:00:00   
model_name unit_no                                           
マイジャグラーV   1095                 4655.0                10.0   
           1096                 7694.0                37.0   
           1097                 2656.0                 7.0   
           1098                 3599.0                15.0   
           1099                 6425.0                22.0   

                                    RB          TOTAL_RATE             RB_RATE  
date               2025-04-14 00:00:00 2025-04-14 00:00:00 2025-04-14 00:00:00  
model_name unit_no                                                              
マイジャグラーV   1095                   14.0               194.0               332.5  
           1096                   25.0               124.1               307.8  
           1097                    8.0               177.1               332.0  
           1098                    7.0               163.6               514.1  
           1099                   17.0               164.7               377.9

In [325]:
SHEET_NAME = "MEDALS_nDAYS_AGO"

today = datetime.date.today()
start_date = today + datetime.timedelta(days=-1)
end_date = today + datetime.timedelta(days=-3)

models = [
    "マイジャグラーV",
    "ゴーゴージャグラー3",
    "アイムジャグラーEX-TP",
    "ファンキージャグラー2",
]

# sheet = spreadsheet.worksheet(SHEET_NAME)
# sheet.clear()

# next_row = 1
# for model in models:
#     medals = get_medals_summary(start_date, end_date, model)
#     set_with_dataframe(sheet, medals, row=next_row, include_index=True)
#     existing = get_as_dataframe(sheet, evaluate_formulas=True)
#     next_row += medals.shape[0] + 5

### 
- 3月5日の過去1日間の出玉ワースト10 → 336以下は5台
- 3月5日の過去2日間の出玉ワースト10 → 336以下は3台
- 3月5日の過去3日間の出玉ワースト10 → 336以下は7台
- 3月5日の過去4日間の出玉ワースト10 → 336以下は6台
### 
- 1月12日の過去1日間の出玉ワースト10 → 336以下は5台
- 1月12日の過去2日間の出玉ワースト10 → 336以下は5台
- 1月12日の過去3日間の出玉ワースト10 → 336以下は5台
- 1月12日の過去4日間の出玉ワースト10 → 336以下は4台
- 1月12日の過去5日間の出玉ワースト10 → 336以下は3台
### 
- 2月12日の過去1日間の出玉ワースト10 → 336以下は3台
- 2月12日の過去2日間の出玉ワースト10 → 336以下は4台
- 2月12日の過去3日間の出玉ワースト10 → 336以下は5台
- 2月12日の過去4日間の出玉ワースト10 → 336以下は4台
- 2月12日の過去5日間の出玉ワースト10 → 336以下は5台
###
- 3月12日の過去1日間の出玉ワースト10 → 336以下は0台
- 3月12日の過去2日間の出玉ワースト10 → 336以下は1台
- 3月12日の過去3日間の出玉ワースト10 → 336以下は3台
- 3月12日の過去4日間の出玉ワースト10 → 336以下は4台
- 3月12日の過去5日間の出玉ワースト10 → 336以下は3台
- 3月12日の過去6日間の出玉ワースト10 → 336以下は3台
- 3月12日の過去7日間の出玉ワースト10 → 336以下は4台

In [47]:
df_tmp = df.copy()
df_tmp[df_tmp["date"].dt.date <= datetime.date(2024, 4, 26)]

,hall_name,date,model_name,unit_no,game,BB,RB,medals,BB_rate,RB_rate,Total_rate,day,month,weekday


In [127]:
print(today - datetime.timedelta(days=347))

2024-05-02
